In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import math
import cv2
import os
import unicodedata
import string
import unidecode
import numpy as np

GATHER DATA

In [2]:
path = "../plot_sums/test"

movie_names = open("{}/movie_names.txt".format(path))
movie_names = movie_names.read().split(",,,")

box_office = open("{}/box_office.txt".format(path))
box_office = box_office.read().split(",,,")

imdbRating = open("{}/movie_ratings.txt".format(path))
imdbRating = imdbRating.read().split(",,,")

metascore = open("{}/metascore.txt".format(path))
metascore = metascore.read().split(",,,")

movie_runtime = open("{}/movie_runtime.txt".format(path))
movie_runtime = movie_runtime.read().split(",,,")
real_runtimes = []
for runtime in movie_runtime:
    real_runtimes.append(runtime.replace(" min", ""))
movie_runtime = real_runtimes

movieBudget = open("{}/movieBudget.txt".format(path))
movieBudget = movieBudget.read().split(",,,")

movieCast = open("{}/movie_cast.txt".format(path))
movieCast = movieCast.read().split(",,,")

movie_directors = open("{}/directors.txt".format(path))
movie_directors = movie_directors.read().split(",,,")

mpaa = open("{}/movie_mpaa.txt".format(path))
mpaa = mpaa.read().split(",,,")

genre = open("{}/movie_genre.txt".format(path))
genre = genre.read().split(",,,")

In [3]:
movie_names = movie_names[:-2001]
print("Movies: {}".format(len(movie_names)))

box_office = box_office[:-2001]
print("Box Office: {}".format(len(box_office)))

imdbRating = imdbRating[:-2001]
print("IMDb Rating: {}".format(len(imdbRating)))

metascore = metascore[:-2001]
print("Metascore: {}".format(len(metascore)))

movie_runtime = movie_runtime[:-2001]
print("Runtime: {}".format(len(movie_runtime)))

movieBudget = movieBudget[:-2001]
print("Budget: {}".format(len(movieBudget)))

movieCast = movieCast[:-2001]
print("Movie Cast: {}".format(len(movieCast)))

genre = genre[:-2001]
genre_rating =[]
for x in genre:
    for idx, val in enumerate(set(genre)):
        if x == val:
            genre_rating.append(idx)
genre = genre_rating
print("Genre: {}".format(len(genre)))

mpaa = mpaa[:-2001]
print("MPAA: {}".format(len(mpaa)))

Movies: 13000
Box Office: 13000
IMDb Rating: 13000
Metascore: 13000
Runtime: 13000
Budget: 13000
Movie Cast: 13000
Genre: 13000
MPAA: 13000


In [4]:
df = pd.DataFrame({
    'name': movie_names,
    'box_office': box_office,
    'rating': imdbRating,
    'metascore': metascore,
    'runtime': movie_runtime,
    'budget': movieBudget,
    'genre': genre,
    'mpaa':mpaa
})
df.head(50)

,name,box_office,rating,metascore,runtime,budget,genre,mpaa
0,Star Wars: Episode VII - The Force Awakens,936662225,8.0,81,136,245000000,57,PG-13
1,Avengers: Endgame,851612415,8.7,78,181,356000000,57,PG-13
2,Avatar,760507625,7.8,83,162,237000000,404,PG-13
3,Black Panther,700059566,7.3,88,134,200000000,57,PG-13
4,Avengers: Infinity War,678815482,8.5,68,149,321000000,57,PG-13
5,Titanic,659325379,7.8,75,194,200000000,349,PG-13
6,Jurassic World,652270625,7.0,59,124,150000000,57,PG-13
7,The Avengers,623357910,8.1,69,143,220000000,57,PG-13
8,Star Wars: Episode VIII - The Last Jedi,620181382,7.1,85,152,317000000,404,PG-13
9,Incredibles 2,608581744,7.7,80,118,200000000,512,PG


In [5]:
df.drop(df[df['budget'] == 'null'].index, inplace = True) 
df.drop(df[df['runtime'] == 'null'].index, inplace = True) 
df.drop(df[df['box_office'] == 'null'].index, inplace = True) 
df.drop(df[df['metascore'] == 'null'].index, inplace = True) 
df.drop(df[df['rating'] == 'null'].index, inplace = True) 
df.drop(df[df['mpaa'] == 'null'].index, inplace = True) 
# df.drop(df[df['budget'].str.contains('£')].index, inplace = True) 
# df.drop(df[df['budget'].str.contains('€')].index, inplace = True) 
# df.drop(df[df['budget'].str.contains('\xa3')].index, inplace = True) 
# # df.drop(df[df['budget'].str.contains('\x')].index, inplace = True) 
# df.drop(df[df['budget'].str.contains('\x8059660000')].index, inplace = True)
# df.drop(df[df['budget'].str.contains('\x8017000000')].index, inplace = True)
# df.drop(df[df['budget'].str.contains('[A-Za-z]', regex=True)].index, inplace = True) 


In [6]:
len(set(mpaa))
print(set(df['mpaa']))

{'GP', 'Approved', 'R', 'Passed', 'Not Rated', 'M', 'G', 'TV-MA', 'TV-PG', 'M/PG', 'Unrated', 'NC-17', 'PG', 'PG-13'}


In [7]:
mpaa_rating =[]
for x in df['mpaa']:
    for idx, val in enumerate(set(df['mpaa'])):
        if x == val:
            mpaa_rating.append(idx)
new_office = []
for value in df['box_office']:
    new_office.append(float(value))
    
new_budget = []
for value in df['budget']:
    new_budget.append(float(value))

In [8]:
print("Name: {}".format(len(df['name'])))
print("BO: {}".format(len(new_office)))
print("Rating: {}".format(len(df['rating'])))
print("Meta: {}".format(len(df['metascore'])))
print("Runtime: {}".format(len(df['runtime'])))
print("Budget: {}".format(len(df['budget'])))
print("Genre: {}".format(len(df['genre'])))

Name: 5538
BO: 5538
Rating: 5538
Meta: 5538
Runtime: 5538
Budget: 5538
Genre: 5538


In [9]:
df = pd.DataFrame({
    'name': df['name'],
    'box_office': new_office,
    'rating': df['rating'],
    'metascore': df['metascore'],
    'runtime': df['runtime'],
    'budget': new_budget,
    'genre': df['genre'],
    'mpaa': mpaa_rating
})

In [10]:
df.head(50)

,name,box_office,rating,metascore,runtime,budget,genre,mpaa
0,Star Wars: Episode VII - The Force Awakens,936662225.0,8.0,81,136,245000000.0,57,13
1,Avengers: Endgame,851612415.0,8.7,78,181,356000000.0,57,13
2,Avatar,760507625.0,7.8,83,162,237000000.0,404,13
3,Black Panther,700059566.0,7.3,88,134,200000000.0,57,13
4,Avengers: Infinity War,678815482.0,8.5,68,149,321000000.0,57,13
5,Titanic,659325379.0,7.8,75,194,200000000.0,349,13
6,Jurassic World,652270625.0,7.0,59,124,150000000.0,57,13
7,The Avengers,623357910.0,8.1,69,143,220000000.0,57,13
8,Star Wars: Episode VIII - The Last Jedi,620181382.0,7.1,85,152,317000000.0,404,13
9,Incredibles 2,608581744.0,7.7,80,118,200000000.0,512,12


In [11]:
X = df[["metascore", "runtime", "rating", "budget", "genre", "mpaa"]]

In [12]:
# for value in df['box_office']:
#     print(type(value))

In [13]:
y_train_regression = df['box_office']
y_train_regression= y_train_regression.astype(np.float)
len(y_train_regression)

5538

In [14]:
X['metascore'] = X['metascore'].astype(np.float)
X['budget'] = X['budget'].astype(np.float)
X['runtime'] = X['runtime'].astype(np.float)
X['rating'] = X['rating'].astype(np.float)
X['genre'] = X['genre'].astype(np.float)
X['mpaa'] = X['mpaa'].astype(np.float)

X.head()
print("Meta: {}".format(len(X['metascore'])))
print("Budget: {}".format(len(X['budget'])))
print("Runtime: {}".format(len(X['runtime'])))
print("Rating: {}".format(len(X['rating'])))
print("Genre: {}".format(len(X['genre'])))
print("Mpaa: {}".format(len(X['mpaa'])))

Meta: 5538
Budget: 5538
Runtime: 5538
Rating: 5538
Genre: 5538
Mpaa: 5538


c:\users\brian\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\brian\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\brian\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [15]:
y_train_regression.head()

0    936662225.0
1    851612415.0
2    760507625.0
3    700059566.0
4    678815482.0
Name: box_office, dtype: float64

In [16]:
y = []
for value in y_train_regression:
    value = float(value)
#     print (type(value))
    if value < 1000000:
        y.append(0)
    elif value > 1000000 and value <= 10000000:
        y.append(1)
    elif value > 10000000 and value <= 25000000:
        y.append(2)
    elif value > 25000000 and value <= 50000000:
        y.append(3)
    elif value > 50000000 and value <= 100000000:
        y.append(4)
    elif value > 100000000 and value <= 200000000:
        y.append(5)
    elif value > 200000000 and value <= 300000000:
        y.append(6)
    elif value > 300000000 and value <= 400000000:
        y.append(7)
    elif value > 400000000 and value <= 500000000:
        y.append(8)
    elif value > 500000000:
        y.append(9)
    else:
        print("ERROR - {}".format(value))

In [17]:
# y

In [18]:
len(y)

5538

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [20]:
import torch
import torch.utils.data
from torch.autograd import Variable

In [21]:
'''Create tensors for our train and test set. 
As you remember we need variable to accumulate gradients. 
Therefore first we create tensor, then we will create variable '''
# Numpy to Tensor Conversion (Train Set)
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

# Numpy to Tensor Conversion (Train Set)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [22]:
set(y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [23]:
# Make torch datasets from train and test sets
train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

# Create train and test data loaders
train_loader = torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = 64, shuffle = True)

In [24]:
import torch.nn as nn
import torch.nn.functional as F

In [25]:
class ANN(nn.Module):
    def __init__(self, input_dim = 2, output_dim = 10):
        super(ANN, self).__init__()
    
        # Input Layer (2) -> 784
        self.fc1 = nn.Linear(input_dim, 256)
        # 256 -> 128
        self.fc2 = nn.Linear(256, 128)
        # 128 -> 128
        self.fc3 = nn.Linear(128, 128)
        # 128 -> 64
        self.fc4 = nn.Linear(128, 64)
        # 64 -> 64
        self.fc5 = nn.Linear(64, 64)
        # 64 -> 32
        self.fc6 = nn.Linear(64, 32)
        # 32 -> 32
        self.fc7 = nn.Linear(32, 32)
        # 32 -> output layer(10)
        self.output_layer = nn.Linear(32,10)
        # Dropout Layer (20%) to reduce overfitting
        self.dropout = nn.Dropout(0.2)
    
    # Feed Forward Function
    def forward(self, x):
        
        # csv input
        x = x.view(-1, 2)
        
        # Add ReLU activation function to each layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        # Add dropout layer
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.dropout(x)
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.dropout(x)
        # Don't add any ReLU activation function to Last Output Layer
        x = self.output_layer(x)
        
        # Return the created model
        return x

In [26]:
# Create the Neural Network Model
model = ANN(input_dim = 2, output_dim = 10)
# Print its architecture
print(model)

ANN(
  (fc1): Linear(in_features=2, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=32, bias=True)
  (fc7): Linear(in_features=32, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=10, bias=True)
  (dropout): Dropout(p=0.2)
)


In [27]:
import torch.optim as optim
# specify loss function
loss_fn = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9,nesterov = True)

In [28]:
# Define epochs (between 20-50)
epochs = 30

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

# Some lists to keep track of loss and accuracy during each epoch
epoch_list = []
train_loss_list = []
val_loss_list = []
train_acc_list = []
val_acc_list = []



# Start epochs
for epoch in range(epochs):
    # monitor training loss
    train_loss = 0.0
    val_loss = 0.0
    
    ###################
    # train the model #
    ###################
    # Set the training mode ON -> Activate Dropout Layers
    model.train() # prepare model for training
    # Calculate Accuracy         
    correct = 0
    total = 0
    
    # Load Train Images with Labels(Targets)
    for data, target in train_loader:
        
        # Convert our feature and labels to Variables to accumulate Gradients
        data = Variable(data).float()
        target = Variable(target).type(torch.LongTensor)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # Calculate Training Accuracy 
        predicted = torch.max(output.data, 1)[1]        
        # Total number of labels
        total += len(target)
        # Total correct predictions
        correct += (predicted == target).sum()
        
        # calculate the loss
        loss = loss_fn(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
    
    # calculate average training loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    
    # Avg Accuracy
    accuracy = 100 * correct / float(total)
    
    # Put them in their list
    train_acc_list.append(accuracy)
    train_loss_list.append(train_loss)
    
        
    # Implement Validation like K-fold Cross-validation 
    # Set Evaluation Mode ON -> Turn Off Dropout
    model.eval() # Required for Evaluation/Test

    # Calculate Test/Validation Accuracy         
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Convert our images and labels to Variables to accumulate Gradients
            data = Variable(data).float()
            target = Variable(target).type(torch.LongTensor)

            # Predict Output
            output = model(data)

            # Calculate Loss
            loss = loss_fn(output, target)
            val_loss += loss.item()*data.size(0)
            # Get predictions from the maximum value
            predicted = torch.max(output.data, 1)[1]

            # Total number of labels
            total += len(target)

            # Total correct predictions
            correct += (predicted == target).sum()
    
    # calculate average training loss and accuracy over an epoch
    val_loss = val_loss/len(test_loader.dataset)
    accuracy = 100 * correct/ float(total)
    
    # Put them in their list
    val_acc_list.append(accuracy)
    val_loss_list.append(val_loss)
    
    # Print the Epoch and Training Loss Details with Validation Accuracy   
    print('Epoch: {} \tTraining Loss: {:.4f}\t Val. acc: {:.2f}%'.format(
        epoch+1, 
        train_loss,
        accuracy
        ))
    # save model if validation loss has decreased
    if val_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        val_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = val_loss
    # Move to next epoch
    epoch_list.append(epoch + 1)

RuntimeError: The size of tensor a (192) must match the size of tensor b (64) at non-singleton dimension 0